In [1]:
import os
import json
import pandas as pd
from urllib.request  import urlretrieve
import gzip
import folium 

In [2]:
##透過這個網址可以下載最新的資料
data_url='https://pm25.lass-net.org/data/last-all-airbox.json.gz'
##開始下載資料 後面是放存的路徑
urlretrieve(data_url,r'D:\pm2.5\last-all-airbox.json.gz')

('D:\\pm2.5\\last-all-airbox.json.gz',
 <http.client.HTTPMessage at 0x1ebc800df88>)

In [3]:
##自動讀取壓縮檔的json檔
with gzip.open(r'D:\pm2.5\last-all-airbox.json.gz','rb') as f:
    last_data=f.read()
    data = json.loads(last_data)

#### 欄位說明  

| s_d0 | s_d1 | s_d2 |
|---| ---| --- |
| PM2.5 | PM 10 | PM1|

| s_t0 | s_h0 |
| ---| --- |
|Temperature | Humidity |


In [4]:
## 原始資料 可以觀察是在feed裡面
#data
data_list=data['feeds']
##一個測站的數值
data_list[10]

{'gps_num': 9.0,
 'gps_fix': 1.0,
 'gps_alt': 2.0,
 'app': 'AirBox',
 'SiteName': '萬華-2',
 'SiteAddr': 'Wanhua',
 's_d0': 26.0,
 's_d1': 28.0,
 's_h0': 100.0,
 'area': 'taipei',
 'gps_lat': 25.046,
 's_t0': 29.75,
 'gps_lon': 121.508,
 'date': '2020-08-25',
 'device_id': '08BEAC028690',
 'name': '手動站：萬華站-2（690)',
 'time': '14:51:43',
 's_d2': 18.0,
 'timestamp': '2020-08-25T14:51:43Z',
 'c_d0': 15.11,
 'c_d0_method': 'SVR/30/0.11'}

In [5]:
##存資料的DataFrame
data_taipei=pd.DataFrame(columns=['device_id','PM2.5','lat','lon'])
## i存沒有區域資訊的站有幾個
i=0
for i in range(len(data_list)):
##抓出在台北的測站
    try:
        if data_list[i]['area']=='taipei':
            data_taipei.loc[i,'device_id']=data_list[i]['device_id']
            data_taipei.loc[i,'PM2.5']=data_list[i]['s_d0']
            data_taipei.loc[i,'lat']=data_list[i]['gps_lat']
            data_taipei.loc[i,'lon']=data_list[i]['gps_lon']
        
    except:
        i=i+1
print(i)
data_taipei.reset_index(inplace=True)
data_taipei.head()

3048


,index,device_id,PM2.5,lat,lon
0,10,08BEAC028690,26,25.046,121.508
1,46,08BEAC028780,15,25.119,121.469
2,53,08BEAC0287F6,26,25.038,121.546
3,138,08BEAC028A20,16,24.8211,121.103
4,152,08BEAC028A4C,25,25.0417,121.544


In [6]:
#畫一張可以知道各站點位置的地圖 還有pm2.5的值
m = folium.Map( location=[25.04282, 121.538],zoom_start=13,tiles='OpenStreetMap')

tooltip = 'Click me!'
for i in range(len(data_taipei)):
    df = pd.DataFrame(data=[[data_taipei['device_id'][i],data_taipei['PM2.5'][i]]], columns=['ID','PM2.5'])
    html = df.to_html(classes='table table-striped table-hover table-condensed table-responsive')
    popup = folium.Popup(html)
    folium.Marker([data_taipei.lat[i],data_taipei.lon[i]], popup=popup, tooltip=tooltip,icon=folium.Icon(color='red')).add_to(m)
m

In [7]:
##拿測站pm2.5資料畫熱點圖
from folium import plugins
from folium.plugins import HeatMap
##生成資料
PM=data_taipei.apply(lambda x: [x['lat'] ,x['lon'],x['PM2.5']], axis=1)

m = folium.Map([25.04282, 121.538], tiles='OpenStreetMap', zoom_start=13)

HeatMap(PM).add_to(folium.FeatureGroup(name='Heat Map').add_to(m))
folium.LayerControl().add_to(m)

m